In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"


In [2]:

from numba import njit
import numpy as np
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="5m",
)

It took 00 mins and 00 seconds to download 200 candles


In [4]:
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product


mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([3]),
    max_trades=np.array([5]),
    num_candles=np.array([61]),
    risk_account_pct_size=np.array([0.2, 5]),
    risk_reward=np.array([64.5]),
    sl_based_on_add_pct=np.array([2]),
    sl_based_on_lookback=np.array([6]),
    sl_bcb_type=np.array([2]),
    sl_to_be_cb_type=np.array([1.948, 6]),
    sl_to_be_when_pct=np.array([1000]),
    sl_to_be_ze_type=np.array([54.61]),
    static_leverage=np.array([153]),
    trail_sl_bcb_type=np.array([0.64654]),
    trail_sl_by_pct=np.array([1.20]),
    trail_sl_when_pct=np.array([156.6]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = nb_dos_cart_product(
    dos_arrays=dos_arrays,
)

In [5]:
from nb_quantfreedom.nb_base import backtest_df_only
from nb_quantfreedom.nb_custom_logger import nb_WillLog
from nb_quantfreedom.strategies.strategy_example import StrategyExample


strategy = StrategyExample(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    rsi_length=[14],
    rsi_is_below=[50],
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    dos_cart_arrays=dos_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=mufex_main.exchange_settings,
    strategy=strategy,
    candles=candles,
    static_os=static_os,
)

TypeError: backtest_df_only() missing 1 required positional argument: 'logger'